In [69]:
import numpy as np

In [70]:
class function:
    def __init__(self,function_as_real_map):
        self.real_map = function_as_real_map
    def function_as_discrete_map(self, start, end, points_num):
        return [self.real_map(start+i*(end-start)/(points_num-1)) for i in range(points_num)]
    def function_as_discrete_map_extra_points(self, start, end, points_num):
        return self.function_as_discrete_map(start, end+5, points_num+5)

def laplace_integration(list):
    k_7_0 = 41393.0/60480.0
    k_7_1 = -23719.0/60480.0
    k_7_2 = 22742.0/60480.0
    k_7_3 = -14762.0/60480.0
    k_7_4 = 5449.0/60480.0
    k_7_5 = -863.0/60480.0

    k_7 = [k_7_0, k_7_1, k_7_2, k_7_3, k_7_4, k_7_5]
    
    b_coor = len(list) - 6
    result = sum([list[i] for i in range(0,b_coor)])
    result += sum([k_7[i] * list[b_coor + i] for i in range(0,6)])
    result -= sum([k_7[i] * list[i] for i in range(0,6)])
    
    return result


In [71]:
x_squared = function(lambda x: x*x)

In [72]:
grid_values = x_squared.function_as_discrete_map_extra_points(0.0,10.0,11)
grid_values

[0.0,
 1.0,
 4.0,
 9.0,
 16.0,
 25.0,
 36.0,
 49.0,
 64.0,
 81.0,
 100.0,
 121.0,
 144.0,
 169.0,
 196.0,
 225.0]

In [73]:
laplace_integration(grid_values)

333.33333333333337

In [74]:
import scipy.integrate as integrate
integrate.quad(x_squared.real_map, 0, 10)

(333.33333333333326, 3.700743415417188e-12)

In [75]:
import scipy.integrate as integrate

def compare_integration_methods(integrand_as_lambda,start,end,num_point_evals):
    f = function(integrand_as_lambda)
    grid_values = f.function_as_discrete_map_extra_points(start,end,num_point_evals)
    laplace_int = laplace_integration(grid_values)
    theoretical_int, theoretical_dev = integrate.quad(f.real_map, start, end)
    abs_dev = np.abs(theoretical_int-laplace_int)
    rel_dev = np.abs(theoretical_int-laplace_int)/theoretical_int
    print("Percentage deviation:", rel_dev)
    print("Absolute deviation:", abs_dev)
    print("Uncertainty in theoretical value:", theoretical_dev)

def compare_integration_methods_verbose(integrand_as_lambda,start,end,num_point_evals):
    f = function(integrand_as_lambda)
    grid_values = f.function_as_discrete_map_extra_points(start,end,num_point_evals)
    laplace_int = laplace_integration(grid_values)
    theoretical_int, theoretical_dev = integrate.quad(f.real_map, start, end)
    abs_dev = np.abs(theoretical_int-laplace_int)
    rel_dev = np.abs(theoretical_int-laplace_int)/theoretical_int
    print("Theoretical value:", theoretical_int)
    print(f"Interval from {start} to {end}")
    print("Percentage deviation:", rel_dev)
    print("Absolute deviation:", abs_dev)
    print("Uncertainty in theoretical value:", theoretical_dev)
    print("--------------------------------------------------")


In [76]:
compare_integration_methods(lambda x: x*x,0.0,10.0,11)

Percentage deviation: 3.41060513165e-16
Absolute deviation: 1.13686837722e-13
Uncertainty in theoretical value: 3.700743415417188e-12


In [77]:
# Example with annuity of a zero year old - intentisty constant values from Norberg eq. (3.25)
interest_rate = 0.02 # arbitrary choice 
alpha = 0.0005
beta = 7.5858e-05
gamma = np.math.log(1.09144)
p_survival_x_year_old = lambda t,x: np.math.exp(-alpha*t-beta*(np.math.exp(gamma*(t+x))-1)/gamma)
p_survival_zero_year_old = lambda t: p_survival_x_year_old(t,0)
annuity_x_year_old_integrand = lambda t,x: np.math.exp(-1*interest_rate*t)*p_survival_x_year_old(t,x)
annuity_zero_year_old_integrand = lambda t: np.math.exp(-1*interest_rate*t)*p_survival_zero_year_old(t)

function(annuity_zero_year_old_integrand).function_as_discrete_map(0,120,12)

[1.0,
 0.7984992210323577,
 0.6361919882852219,
 0.5039718605924449,
 0.3933165453538113,
 0.29528485238517843,
 0.20045054639867776,
 0.10475631057767379,
 0.027752547646899984,
 0.001259078477171793,
 5.83717348811404e-07,
 1.8277053679093564e-15]

In [78]:
compare_integration_methods(annuity_zero_year_old_integrand,0.0,120.0,121)

Percentage deviation: 3.78374019353e-15
Absolute deviation: 1.42108547152e-13
Uncertainty in theoretical value: 3.395439631872089e-10


In [79]:
# Examples with basic functions
print("x cubed")
compare_integration_methods(lambda x: x**3,0.0,10.0,11)
print("x to the power 5")
compare_integration_methods(lambda x: x**5,0.0,10.0,11)
print("x to the power 6")
compare_integration_methods(lambda x: x**6,0.0,10.0,11)
print("x to the power 7")
compare_integration_methods(lambda x: x**7,0.0,10.0,11)
print("x to the power 8")
compare_integration_methods(lambda x: x**8,0.0,10.0,11)
print("exponential")
compare_integration_methods(lambda x: np.exp(x),0.0,10.0,11)
print("exponential")
compare_integration_methods(lambda x: np.sin(x),0.0,10.0,11)

x cubed
Percentage deviation: 1.81898940355e-16
Absolute deviation: 4.54747350886e-13
Uncertainty in theoretical value: 2.775557561562892e-11
x to the power 5
Percentage deviation: 0.0
Absolute deviation: 0.0
Uncertainty in theoretical value: 1.8503717077085944e-09
x to the power 6
Percentage deviation: 1.62981450558e-16
Absolute deviation: 2.32830643654e-10
Uncertainty in theoretical value: 1.5860328923216523e-08
x to the power 7
Percentage deviation: 4.58333333334e-05
Absolute deviation: 572.916666668
Uncertainty in theoretical value: 1.387778780781446e-07
x to the power 8
Percentage deviation: 0.000296047
Absolute deviation: 32894.1111111
Uncertainty in theoretical value: 1.2335811384723963e-06
exponential
Percentage deviation: 0.122354406265
Absolute deviation: 2694.91279003
Uncertainty in theoretical value: 6.239389118119916e-10
exponential
Percentage deviation: 0.0061622947631
Absolute deviation: 0.0113329008526
Uncertainty in theoretical value: 9.92116330910929e-12


In [80]:
# Examples with different x for annuities
print("For a 80 year old doing the integral from 0 to 120")
compare_integration_methods_verbose(lambda t: annuity_x_year_old_integrand(t,80),0.0,120.0,121)
print("For a 80 year old doing the integral from 0 to 30")
compare_integration_methods_verbose(lambda t: annuity_x_year_old_integrand(t,80),0.0,30.0,31)
print("For a 100 year old doing the integral from 0 to 120")
compare_integration_methods_verbose(lambda t: annuity_x_year_old_integrand(t,100),0.0,120.0,121)
print("For a 100 year old doing the integral from 0 to 30")
compare_integration_methods_verbose(lambda t: annuity_x_year_old_integrand(t,100),0.0,30.0,31)

For a 80 year old doing the integral from 0 to 120
Theoretical value: 2.458672942142802
Interval from 0.0 to 120.0
Percentage deviation: 3.37622794691e-09
Absolute deviation: 8.30104029959e-09
Uncertainty in theoretical value: 2.025232637842267e-10
--------------------------------------------------
For a 80 year old doing the integral from 0 to 30
Theoretical value: 2.458672075352603
Interval from 0.0 to 30.0
Percentage deviation: 4.16876335106e-09
Absolute deviation: 1.024962204e-08
Uncertainty in theoretical value: 1.0727505925202197e-12
--------------------------------------------------
For a 100 year old doing the integral from 0 to 120
Theoretical value: 0.007348811477706047
Interval from 0.0 to 120.0
Percentage deviation: 5.11575113186e-06
Absolute deviation: 3.75946906349e-08
Uncertainty in theoretical value: 4.145520167647774e-09
--------------------------------------------------
For a 100 year old doing the integral from 0 to 30
Theoretical value: 0.007348811477706047
Interval